In [2]:
import numpy as np
import torch
from torchvision import transforms
import cv2
from PIL import Image, ImageDraw
from CFA import CFA

In [42]:
# param
num_landmark = 24
img_width = 128
checkpoint_name = 'checkpoint_landmark_191116.pth'
input_img_name = '/home/jempio/documents/aps360-project/data/dataSetB_10k/36253_2011.jpg'

In [43]:
import cv2
import numpy as np
from PIL import Image, ImageDraw
import torch
import torchvision.transforms as transforms

# Assuming CFA and other necessary imports are already available
# detector
face_detector = cv2.CascadeClassifier('lbpcascade_animeface.xml')
landmark_detector = CFA(output_channel_num=num_landmark + 1, checkpoint_name=checkpoint_name).cuda()

# transform
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
train_transform = [transforms.ToTensor(), normalize]
train_transform = transforms.Compose(train_transform)

# input image & detect face
img = cv2.imread(input_img_name)
if img is None:
    raise FileNotFoundError(f"Unable to load image at {input_img_name}")
faces = face_detector.detectMultiScale(img)

# Create a black image with the same size
black_img = np.zeros_like(img)
black_img = Image.fromarray(black_img)  # Initialize with black background
draw = ImageDraw.Draw(black_img)

# Convert the original image to PIL Image for cropping
img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

for x_, y_, w_, h_ in faces:

    # adjust face size
    x = max(x_ - w_ / 8, 0)
    rx = min(x_ + w_ * 9 / 8, black_img.width)
    y = max(y_ - h_ / 4, 0)
    by = y_ + h_
    w = rx - x
    h = by - y

    # draw result of face detection
    draw.rectangle((x, y, x + w, y + h), outline=(255, 255, 255), width=3)

    # transform image
    img_tmp = img_pil.crop((x, y, x+w, y+h))
    img_tmp = img_tmp.resize((img_width, img_width), Image.BICUBIC)
    img_tmp = train_transform(img_tmp)
    img_tmp = img_tmp.unsqueeze(0).cuda()

    # estimate heatmap
    heatmaps = landmark_detector(img_tmp)
    heatmaps = heatmaps[-1].cpu().detach().numpy()[0]

    landmarks = []

    # calculate landmark position
    for i in range(num_landmark):
        heatmaps_tmp = cv2.resize(heatmaps[i], (img_width, img_width), interpolation=cv2.INTER_CUBIC)
        landmark = np.unravel_index(np.argmax(heatmaps_tmp), heatmaps_tmp.shape)
        landmark_y = landmark[0] * h / img_width
        landmark_x = landmark[1] * w / img_width

        landmarks.append((x + landmark_x, y + landmark_y))

        # draw landmarks
        draw.ellipse((x + landmark_x - 2, y + landmark_y - 2, x + landmark_x + 2, y + landmark_y + 2), fill=(255, 255, 255))

    # draw boxes around eyes, nose, and mouth
    # Adjust indices based on specific model
    eyebrow_left_indices = [3, 4, 5]
    eyebrow_right_indices = [6, 7, 8]
    eye_left_indices = [0, 10, 11, 12, 13, 14]
    eye_right_indices = [2, 15, 16, 17, 18, 19]
    nose_indices = [9]
    mouth_indices = [1, 20, 22, 23]

    def get_bounding_box(points):
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        return (min(x_coords), min(y_coords), max(x_coords), max(y_coords))

    if all(i < num_landmark for i in eyebrow_left_indices):
        eyebrow_left_points = [landmarks[i] for i in eyebrow_left_indices]
        draw.rectangle(get_bounding_box(eyebrow_left_points), outline=(0, 255, 0), width=2)

    if all(i < num_landmark for i in eyebrow_right_indices):
        eyebrow_right_points = [landmarks[i] for i in eyebrow_right_indices]
        draw.rectangle(get_bounding_box(eyebrow_right_points), outline=(0, 255, 0), width=2)

    if all(i < num_landmark for i in eye_left_indices):
        eye_left_points = [landmarks[i] for i in eye_left_indices]
        draw.rectangle(get_bounding_box(eye_left_points), outline=(0, 255, 0), width=2)

    if all(i < num_landmark for i in eye_right_indices):
        eye_right_points = [landmarks[i] for i in eye_right_indices]
        draw.rectangle(get_bounding_box(eye_right_points), outline=(0, 255, 0), width=2)

    if all(i < num_landmark for i in nose_indices):
        nose_points = [landmarks[i] for i in nose_indices]
        draw.rectangle(get_bounding_box(nose_points), outline=(0, 255, 0), width=2)

    if all(i < num_landmark for i in mouth_indices):
        mouth_points = [landmarks[i] for i in mouth_indices]
        draw.rectangle(get_bounding_box(mouth_points), outline=(0, 255, 0), width=2)

# output image
black_img.save('oujutput.bmp')


In [33]:
# import cv2
# import numpy as np
# from PIL import Image, ImageDraw
# import torch
# import torchvision.transforms as transforms

# # Assuming CFA and other necessary imports are already available
# # detector
# face_detector = cv2.CascadeClassifier('lbpcascade_animeface.xml')
# landmark_detector = CFA(output_channel_num=num_landmark + 1, checkpoint_name=checkpoint_name).cuda()

# # transform
# normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# train_transform = [transforms.ToTensor(), normalize]
# train_transform = transforms.Compose(train_transform)

# # input image & detect face
# img = cv2.imread(input_img_name)
# faces = face_detector.detectMultiScale(img)

# # Create a black image with the same size
# black_img = np.zeros_like(img)
# black_img = Image.fromarray(cv2.cvtColor(black_img, cv2.COLOR_BGR2RGB))
# draw = ImageDraw.Draw(black_img)

# # Convert the original image to PIL Image for cropping
# img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# for x_, y_, w_, h_ in faces:

#     # adjust face size
#     x = max(x_ - w_ / 8, 0)
#     rx = min(x_ + w_ * 9 / 8, black_img.width)
#     y = max(y_ - h_ / 4, 0)
#     by = y_ + h_
#     w = rx - x
#     h = by - y

#     # draw result of face detection
#     draw.rectangle((x, y, x + w, y + h), outline=(255, 255, 255), width=3)

#     # transform image
#     img_tmp = img_pil.crop((x, y, x+w, y+h))
#     img_tmp = img_tmp.resize((img_width, img_width), Image.BICUBIC)
#     img_tmp = train_transform(img_tmp)
#     img_tmp = img_tmp.unsqueeze(0).cuda()

#     # estimate heatmap
#     heatmaps = landmark_detector(img_tmp)
#     heatmaps = heatmaps[-1].cpu().detach().numpy()[0]

#     # calculate landmark position
#     for i in range(num_landmark):
#         heatmaps_tmp = cv2.resize(heatmaps[i], (img_width, img_width), interpolation=cv2.INTER_CUBIC)
#         landmark = np.unravel_index(np.argmax(heatmaps_tmp), heatmaps_tmp.shape)
#         landmark_y = landmark[0] * h / img_width
#         landmark_x = landmark[1] * w / img_width

#         # draw landmarks
#         draw.ellipse((x + landmark_x - 2, y + landmark_y - 2, x + landmark_x + 2, y + landmark_y + 2), fill=(255, 255, 255))

#          # label landmarks
#         draw.text((x + landmark_x + 5, y + landmark_y), str(i), fill=(255, 255, 255))


# # output image
# black_img.save('output.bmp')
